# Append, Concat, & Merge 

Estas tres funciones de pandas son de suma importancia al trabajar con varias bases de datos a la vez, son las herramientas de las que haremos uso al tener que unir `DataFrames`, y haremos un repaso en como usar estas funciones.

## Append

`Append` a mi pareces es la más intuitiva y simple de las tres, esta la usamos con el fin de agregar columnas a nuestro `DataFrame`

In [1]:
import pandas as pd
import numpy as np

Crearemos dos dataframes que tengan las mismas columnas

In [2]:
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df

,A,B
0,1,2
1,3,4


In [3]:
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))
df.append(df2)

,A,B
0,1,2
1,3,4
0,5,6
1,7,8


Podemos ver que las filas fueron agregadas al final, notemos que el indice no se cambió, por lo que tenemos indices repetidos

In [4]:
df.append(df2).loc[0]

,A,B
0,1,2
0,5,6


Pandas puede manejar esto, y al invocar ese indice nos entrega ambos valores, pero en general queremos que el indice sea único para cada valor, para esto podemos usar `ignore_index=True`, que hace lo que dice en el nombre

In [5]:
df.append(df2, ignore_index=True)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8


Tambien es bueno saber que, si usamos `append` para `DataFrames` con columnas distintas, los valores "faltantes" seran rellenados con `NaN`, en ambas direcciones

In [6]:
df3 = pd.DataFrame([[5, 6, 'a'], [7, 8, 'b']], columns=list('ABC'))
df3

,A,B,C
0,5,6,a
1,7,8,b


In [7]:
df.append(df3, ignore_index=True) #agregando con mas columnas

,A,B,C
0,1,2,NaN
1,3,4,NaN
2,5,6,a
3,7,8,b


In [8]:
df3.append(df, ignore_index=True) #agregando con menos columnas

,A,B,C
0,5,6,a
1,7,8,b
2,1,2,NaN
3,3,4,NaN


## Concat

`Concat` es similar a `Append`, pero más poderoso. En buenas a primeras parece que hacen lo mismo, agregar una `DataFrame` al final de otro

In [9]:
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
pd.concat([s1, s2])

0    a
1    b
0    c
1    d
dtype: object

Y de la misma forma tenemos que usar `ignore_index=True` si es que los datos compartian indices

In [10]:
pd.concat([s1, s2], ignore_index=True)

0    a
1    b
2    c
3    d
dtype: object

Pero la primera gran diferencia es que esta funcion tiene un entendimiento de que los `DataFrames` que estamos juntando son distintos, motivo por el cual podemos entregarle el argunmento `keys`, que nos permite usar indices multiples para referirnos a cada `Dataframe` que usamos

In [11]:
pd.concat([s1, s2], keys=['s1', 's2'])

s1  0    a
    1    b
s2  0    c
    1    d
dtype: object

In [12]:
df1 = pd.DataFrame([['a', 1], ['b', 2]]
                  , columns=['letter', 'number']
                 )
df1

,letter,number
0,a,1
1,b,2


In [13]:
df2 = pd.DataFrame([['c', 3], ['d', 4]]
                   , columns=['letter', 'number']
                  )
df2

,letter,number
0,c,3
1,d,4


In [14]:
df3 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']] 
                   , columns=['letter', 'number', 'animal']
                  )
df3

,letter,number,animal
0,c,3,cat
1,d,4,dog


La otra gran diferencia, es que podemos unir varias `DataFrames` a la vez, y notemos que tambien rellenan los `NaN` 

In [15]:
pd.concat([df1, df2 ,df3], sort=False)

,letter,number,animal
0,a,1,NaN
1,b,2,NaN
0,c,3,NaN
1,d,4,NaN
0,c,3,cat
1,d,4,dog


## Merge

`Merge` es, a mi parecer, la más compleja de las dos. No porque es dificl de entender, sino por el millar de formas que hay de hacerlo. Hoy explicaré las 4 principales: `inner`, `left`, `rigth`, y `outer`, pero hay varias más. Para esto la documentacion de *Pandas* no es lo mejor, así que les dejo esta respuesta de [*StackOverflow*](https://stackoverflow.com/questions/53645882/pandas-merging-101) (que es de donde saqué las imagenes) por si quieres ver el resto de las posibilidades.

Crearemos dos `Dataframes` distintos y veremos cuales son las formas de hacer un `Merge`

### Inner

In [16]:
np.random.seed(0)
left = pd.DataFrame({'key': ['A', 'B', 'C', 'D'], 'value': np.random.randn(4)})    
right = pd.DataFrame({'key': ['B', 'D', 'E', 'F'], 'value': np.random.randn(4)})

<img src="images/inner.png" width="400">

In [17]:
left.merge(right, on='key') 
# inner es el valor por defecto pero podemos ser más especificos 
# left.merge(right, on='key', how='inner')

,key,value_x,value_y
0,B,0.400157,1.867558
1,D,2.240893,-0.977278


### Left

<img src="images/left.png" width="400">

In [18]:
left.merge(right, on='key', how='left')

,key,value_x,value_y
0,A,1.764052,NaN
1,B,0.400157,1.867558
2,C,0.978738,NaN
3,D,2.240893,-0.977278


### Rigth

<img src="images/rigth.png" width="400">

In [19]:
left.merge(right, on='key', how='right')

,key,value_x,value_y
0,B,0.400157,1.867558
1,D,2.240893,-0.977278
2,E,NaN,0.950088
3,F,NaN,-0.151357


### Outer

<img src="images/outer.png" width="400">

In [20]:
left.merge(right, on='key', how='outer')

,key,value_x,value_y
0,A,1.764052,NaN
1,B,0.400157,1.867558
2,C,0.978738,NaN
3,D,2.240893,-0.977278
4,E,NaN,0.950088
5,F,NaN,-0.151357


# Actividad

Como ya es común, usaremos una base de datos que podemos en contrar en Kaggle, esta ves será ["Restaurant Data with Consumer Ratings"](https://www.kaggle.com/uciml/restaurant-data-with-consumer-ratings?select=rating_final.csv). Citando a Kaggle:

*This dataset was used for a study where the task was to generate a top-n list of restaurants according to the consumer preferences and finding the significant features. Two approaches were tested: a collaborative filter technique and a contextual approach: (i) The collaborative filter technique used only one file i.e., rating_final.csv that comprises the user, item and rating attributes. (ii) The contextual approach generated the recommendations using the remaining eight data files.*

Los `.csv` que usaremos están en la carpeta `data`

Para las actividades que veremos, nos interesa tener el *rating* promedio tanto de los restaurantes como de los usuarios, motivo por el cual construriemos estos dos dataframes

In [21]:
df = pd.read_csv('./data/rating_final.csv')
df.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


El primero se los daré construido, el siguiente deben construirlo ustedes

In [22]:
user_mean = (df.drop('placeID',axis=1) #saco la columna de restaurantes
               .groupby('userID') #junto a todos los usurarion
               .apply(lambda x : x.mean()) #le saco el promedio a cada atributo
            )
user_mean.head()

,rating,food_rating,service_rating
userID,,,
U1001,1.111111,1.222222,1.222222
U1002,1.400000,1.400000,1.000000
U1003,1.615385,1.692308,1.461538
U1004,1.875000,1.875000,1.750000
U1005,1.333333,1.444444,1.000000


In [23]:
res_mean = (df.drop('userID',axis=1)
              .groupby('placeID')
              .apply(lambda x : x.mean())
              .drop('placeID', axis=1)
           )
res_mean.head()

,rating,food_rating,service_rating
placeID,,,
132560,0.50,1.00,0.250000
132561,0.75,1.00,1.000000
132564,1.25,1.25,1.500000
132572,1.00,1.00,0.933333
132583,1.00,1.00,1.250000


Esto dejará de ser magia oscura en algún momento, lo prometo! pero de todas formas les dejo una explicación a *grosso modo* de que es lo que se está ahciendo. 
Ultima cosa antes de ir al ejercicio, un metodo muy util cuando trabajamos con dataframes de datos numericos es el metodo `.describe()`, lo podemos ver a continuacion:

In [24]:
res_mean.describe()

,rating,food_rating,service_rating
count,130.000000,130.000000,130.000000
mean,1.179622,1.213914,1.085800
std,0.349354,0.339270,0.334026
min,0.250000,0.250000,0.250000
25%,1.000000,1.000000,0.904167
50%,1.181818,1.244048,1.000000
75%,1.400000,1.400000,1.272727
max,2.000000,2.000000,2.000000


Su funcion es bastante autoexplicativa, nos entrega un resumen con estadisticos que podemos usar.

Ahora tenemos lo que queremos `user_mean` y `res_mean`, entonces ¿Qué podemos hacer con estos datos?. Veamos primero el dataset con mas columnas. 

In [25]:
places = pd.read_csv('./data/geoplaces2.csv')
places.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services'],
      dtype='object')

Como suele ocurrir, muchas de estás comlumnas no vamos a usar así que nos desaremos de ellas 

In [26]:
no_use = ['latitude', 'longitude', 'the_geom_meter','address','city', 'state', 'fax', 'zip','url']
places = places.drop(no_use, axis = 1)
places.head()

,placeID,name,country,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services
0,134999,Kiku Cuernavaca,Mexico,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none
1,132825,puesto de tacos,mexico,No_Alcohol_Served,none,informal,completely,low,familiar,f,open,none
2,135106,El Rinc�n de San Francisco,Mexico,Wine-Beer,only at bar,informal,partially,medium,familiar,f,open,none
3,132667,little pizza Emilio Portes Gil,?,No_Alcohol_Served,none,informal,completely,low,familiar,t,closed,none
4,132613,carnitas_mata,Mexico,No_Alcohol_Served,permitted,informal,completely,medium,familiar,t,closed,none


Mucho más trabajable, para ver el detalle de las columnas, en la carpeta data hay un `README.txt`, este explica que es cada archivo y cuales son sus columnas. Ahora, nos enfocaremos en 3 `alcohol`, `price`, y `dress_code`. Veremos si es que hay grandes diferencias entre cada una de estas opciones y su  valoracion promedio. 

Pero antes, tenemos un ID para cada restaurante, la columna `placeID` , ya que tenemos una sola fila por cada uno, podemos usar esta como indice:

In [27]:
places = places.set_index('placeID')
places.head()

,name,country,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services
placeID,,,,,,,,,,,
134999,Kiku Cuernavaca,Mexico,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none
132825,puesto de tacos,mexico,No_Alcohol_Served,none,informal,completely,low,familiar,f,open,none
135106,El Rinc�n de San Francisco,Mexico,Wine-Beer,only at bar,informal,partially,medium,familiar,f,open,none
132667,little pizza Emilio Portes Gil,?,No_Alcohol_Served,none,informal,completely,low,familiar,t,closed,none
132613,carnitas_mata,Mexico,No_Alcohol_Served,permitted,informal,completely,medium,familiar,t,closed,none


Es claro que para hacer esto de una forma rápida, sería conveniente tener una columna que sea cada *rating*, y para esto usaremos `merge`.   Una cosa que no mencioné durante la explicación de `merge` es que podemos hacer sobre los incides, para esto tenemos que especificar que indice queremos usar, y esto lo hacemos con los parametros `left_index = True` y `right_index = True`. Ahora, ya que tenemos `places` y `res_mean` que tienen el mismo indice, hagamos un merge entre las dos:

In [28]:
places = places.merge(res_mean
                      , left_index = True #fill
                      , right_index = True #fill
                      , how = 'inner' #fill
                     )
places.head()

,name,country,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services,rating,food_rating,service_rating
placeID,,,,,,,,,,,,,,
134999,Kiku Cuernavaca,Mexico,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none,1.600000,1.600000,1.6000
132825,puesto de tacos,mexico,No_Alcohol_Served,none,informal,completely,low,familiar,f,open,none,1.281250,1.343750,0.9375
135106,El Rinc�n de San Francisco,Mexico,Wine-Beer,only at bar,informal,partially,medium,familiar,f,open,none,1.200000,1.200000,1.2000
132667,little pizza Emilio Portes Gil,?,No_Alcohol_Served,none,informal,completely,low,familiar,t,closed,none,1.250000,2.000000,1.2500
132613,carnitas_mata,Mexico,No_Alcohol_Served,permitted,informal,completely,medium,familiar,t,closed,none,1.166667,1.333333,1.0000


Primero revisaremos si el expendio de alcohol afecta los *ratings* que recibe cada restaurante, esta columna puede tomar 3 valores `No_Alcohol_Served`, `Wine-Beer`, y `Full_Bar`, esto lo podemos ver de forma rapida usando el metodo `.unique`

In [29]:
places['alcohol'].unique()

array(['No_Alcohol_Served', 'Wine-Beer', 'Full_Bar'], dtype=object)

Ahora, selecionando solo las columnas `rating`, `food_rating`, y `service_rating`, usa el método `.describe()` para ver si hay variaciones entre los estadisticos de cada tipo de restaurante: 

In [30]:
places[places['alcohol']=='No_Alcohol_Served'][['rating', 'food_rating', 'service_rating']].describe()

,rating,food_rating,service_rating
count,87.000000,87.000000,87.000000
mean,1.148075,1.194730,1.042417
std,0.345414,0.342652,0.327117
min,0.250000,0.250000,0.250000
25%,0.916667,1.000000,0.884444
50%,1.166667,1.166667,1.000000
75%,1.333333,1.400000,1.250000
max,2.000000,2.000000,1.833333


In [31]:
places[places['alcohol']=='Full_Bar'][['rating', 'food_rating', 'service_rating']].describe()

,rating,food_rating,service_rating
count,9.000000,9.000000,9.000000
mean,1.287124,1.218315,1.170311
std,0.275485,0.284492,0.251138
min,0.857143,0.777778,0.777778
25%,1.000000,1.000000,1.000000
50%,1.363636,1.181818,1.200000
75%,1.500000,1.333333,1.333333
max,1.666667,1.750000,1.500000


In [32]:
places[places['alcohol']=='Wine-Beer'][['rating', 'food_rating', 'service_rating']].describe()

,rating,food_rating,service_rating
count,34.000000,34.000000,34.000000
mean,1.231887,1.261840,1.174437
std,0.373067,0.347866,0.356361
min,0.250000,0.250000,0.250000
25%,1.000000,1.151261,1.000000
50%,1.217647,1.278846,1.145833
75%,1.490385,1.421429,1.333333
max,2.000000,2.000000,2.000000


Esto se puede hacer de forma mucho más simple con un `groupby` y un `apply`, lo dejo en tus manos si lo quires intentar

In [33]:
places.groupby('alcohol').apply(lambda x: x.describe())

rating  food_rating  service_rating
alcohol                                                        
Full_Bar          count   9.000000     9.000000        9.000000
                  mean    1.287124     1.218315        1.170311
                  std     0.275485     0.284492        0.251138
                  min     0.857143     0.777778        0.777778
                  25%     1.000000     1.000000        1.000000
                  50%     1.363636     1.181818        1.200000
                  75%     1.500000     1.333333        1.333333
                  max     1.666667     1.750000        1.500000
No_Alcohol_Served count  87.000000    87.000000       87.000000
                  mean    1.148075     1.194730        1.042417
                  std     0.345414     0.342652        0.327117
                  min     0.250000     0.250000        0.250000
                  25%     0.916667     1.000000        0.884444
                  50%     1.166667     1.166667        1.000000
                  75%     1.333333     1.400000        1.250000
                  max     2.000000     2.000000        1.833333
Wine-Beer         count  34.000000    34.000000       34.000000
                  mean    1.231887     1.261840        1.174437
                  std     0.373067     0.347866        0.356361
                  min     0.250000     0.250000        0.250000
                  25%     1.000000     1.151261        1.000000
                  50%     1.217647     1.278846        1.145833
                  75%     1.490385     1.421429        1.333333
                  max     2.000000     2.000000        2.000000

Es claro que esto no es un test de hipotesis, pero de buenas a primeras, ¿Crees que hay una direrencia significativa entre las valoraciones?

**Respuesta**: sin alcohol baja el promedio un poquito.

Ahora, prueba con otro atributo, ¿alguno que te llame la atención? Haz un proceso similar para otra variable que te llame la atención

In [34]:
places.groupby('smoking_area').apply(lambda x: x.describe())

rating  food_rating  service_rating
smoking_area                                               
none          count  70.000000    70.000000       70.000000
              mean    1.171497     1.208648        1.086371
              std     0.381643     0.397504        0.348372
              min     0.250000     0.250000        0.250000
              25%     0.895833     1.000000        0.938419
              50%     1.209957     1.244048        1.000000
              75%     1.356061     1.400000        1.298951
              max     2.000000     2.000000        2.000000
not permitted count  25.000000    25.000000       25.000000
              mean    1.071238     1.175244        0.986184
              std     0.283766     0.277214        0.264571
              min     0.600000     0.600000        0.400000
              25%     0.916667     1.000000        0.857143
              50%     1.000000     1.200000        1.000000
              75%     1.333333     1.333333        1.166667
              max     1.600000     1.700000        1.428571
only at bar   count   2.000000     2.000000        2.000000
              mean    1.377778     1.266667        1.377778
              std     0.251416     0.094281        0.251416
              min     1.200000     1.200000        1.200000
              25%     1.288889     1.233333        1.288889
              50%     1.377778     1.266667        1.377778
              75%     1.466667     1.300000        1.466667
              max     1.555556     1.333333        1.555556
permitted     count   9.000000     9.000000        9.000000
              mean    1.201852     1.331481        1.128704
              std     0.411674     0.211713        0.519199
              min     0.500000     1.000000        0.250000
              25%     1.000000     1.250000        0.833333
              50%     1.166667     1.333333        1.166667
              75%     1.500000     1.400000        1.500000
              max     1.833333     1.750000        1.833333
section       count  24.000000    24.000000       24.000000
              mean    1.291368     1.221073        1.147479
              std     0.268794     0.260724        0.263152
              min     0.750000     0.750000        0.750000
              25%     1.108333     1.000000        0.989583
              50%     1.242647     1.183333        1.085714
              75%     1.500000     1.361111        1.333333
              max     1.750000     1.750000        1.750000

**Respuesta:** Son promedios bastante parecidos.

Ahora, veamos como se relaciona todo con si los restaruantes tienen estacionamiento:

In [35]:
park = pd.read_csv('./data/chefmozparking.csv')
park.head()

,placeID,parking_lot
0,135111,public
1,135110,none
2,135109,none
3,135108,none
4,135107,none


Veamos que valores pueden tomar estos datos

In [36]:
park['parking_lot'].unique()

array(['public', 'none', 'yes', 'valet parking', 'fee', 'street',
       'validated parking'], dtype=object)

Ahora, de la misma forma haremos un merge, pero esta vez no pondremos la fila `placeID` como indice, para mostrar que podemos hacer el merge entre indices y colunas, para esto usamos ` left_on = 'nombre_col'` (o si la columnas que queremos usar esta en lado derecho `rigth_on = ...`). Por tanto, en este merge le tenemos que decir al merge que lo haga en la columna izquierda `placeID` y en la derecha que use el indice

In [37]:
park = park.merge(places
                  ,  left_on = 'placeID'
                  ,  right_index = True
                 )
park.head()

,placeID,parking_lot,name,country,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services,rating,food_rating,service_rating
2,135109,none,Paniroles,?,Wine-Beer,not permitted,informal,no_accessibility,medium,quiet,f,closed,Internet,1.000000,1.250000,0.750000
3,135108,none,Potzocalli,?,No_Alcohol_Served,none,informal,completely,low,familiar,f,closed,none,1.181818,1.181818,1.000000
5,135106,none,El Rinc�n de San Francisco,Mexico,Wine-Beer,only at bar,informal,partially,medium,familiar,f,open,none,1.200000,1.200000,1.200000
7,135104,yes,vips,?,Full_Bar,not permitted,informal,completely,medium,familiar,t,closed,variety,0.857143,1.428571,0.857143
21,135088,public,Cafeteria cenidet,Mexico,No_Alcohol_Served,not permitted,informal,no_accessibility,low,quiet,f,closed,none,1.000000,1.166667,1.000000


Acá, ¿hay algo que te produsca curiosidad?, hay demasiados valores unicos como para revisarlos uno por uno, asi que usa dos o tres que te llamen la atencion y revisa como re comparan entre ellos

In [38]:
park.groupby('smoking_area').apply(lambda x: x.describe()).drop('placeID', axis=1)

rating  food_rating  service_rating
smoking_area                                               
none          count  70.000000    70.000000       70.000000
              mean    1.171497     1.208648        1.086371
              std     0.381643     0.397504        0.348372
              min     0.250000     0.250000        0.250000
              25%     0.895833     1.000000        0.938419
              50%     1.209957     1.244048        1.000000
              75%     1.356061     1.400000        1.298951
              max     2.000000     2.000000        2.000000
not permitted count  25.000000    25.000000       25.000000
              mean    1.071238     1.175244        0.986184
              std     0.283766     0.277214        0.264571
              min     0.600000     0.600000        0.400000
              25%     0.916667     1.000000        0.857143
              50%     1.000000     1.200000        1.000000
              75%     1.333333     1.333333        1.166667
              max     1.600000     1.700000        1.428571
only at bar   count   2.000000     2.000000        2.000000
              mean    1.377778     1.266667        1.377778
              std     0.251416     0.094281        0.251416
              min     1.200000     1.200000        1.200000
              25%     1.288889     1.233333        1.288889
              50%     1.377778     1.266667        1.377778
              75%     1.466667     1.300000        1.466667
              max     1.555556     1.333333        1.555556
permitted     count   9.000000     9.000000        9.000000
              mean    1.201852     1.331481        1.128704
              std     0.411674     0.211713        0.519199
              min     0.500000     1.000000        0.250000
              25%     1.000000     1.250000        0.833333
              50%     1.166667     1.333333        1.166667
              75%     1.500000     1.400000        1.500000
              max     1.833333     1.750000        1.833333
section       count  24.000000    24.000000       24.000000
              mean    1.291368     1.221073        1.147479
              std     0.268794     0.260724        0.263152
              min     0.750000     0.750000        0.750000
              25%     1.108333     1.000000        0.989583
              50%     1.242647     1.183333        1.085714
              75%     1.500000     1.361111        1.333333
              max     1.750000     1.750000        1.750000

In [39]:
park.groupby('dress_code').apply(lambda x: x.describe()).drop('placeID', axis=1)

rating  food_rating  service_rating
dress_code                                               
casual     count   10.000000    10.000000       10.000000
           mean     1.187739     1.264256        1.118116
           std      0.289743     0.260890        0.215693
           min      0.625000     0.875000        0.880000
           25%      1.000000     1.118235        0.925000
           50%      1.200980     1.320513        1.083333
           75%      1.397222     1.397222        1.280769
           max      1.600000     1.700000        1.444444
formal     count    2.000000     2.000000        2.000000
           mean     1.916667     1.750000        1.916667
           std      0.117851     0.353553        0.117851
           min      1.833333     1.500000        1.833333
           25%      1.875000     1.625000        1.875000
           50%      1.916667     1.750000        1.916667
           75%      1.958333     1.875000        1.958333
           max      2.000000     2.000000        2.000000
informal   count  118.000000   118.000000      118.000000
           mean     1.166441     1.200562        1.068978
           std      0.344271     0.339554        0.327328
           min      0.250000     0.250000        0.250000
           25%      1.000000     1.000000        0.895833
           50%      1.180195     1.200000        1.000000
           75%      1.390909     1.393750        1.262500
           max      2.000000     2.000000        1.800000

**Respuesta:** Los lugares donde se viste formal tienen un mayor promedio.

Ahora lo dejo en tus manos, aun no hemos tocado a los usuarios, pero eso te lo dejo a ti. Tenemos la base de datos `userprofile.csv`, que es muy interesante:

In [40]:
profile = pd.read_csv('./data/userprofile.csv')
profile.columns

Index(['userID', 'latitude', 'longitude', 'smoker', 'drink_level',
       'dress_preference', 'ambience', 'transport', 'marital_status', 'hijos',
       'birth_year', 'interest', 'personality', 'religion', 'activity',
       'color', 'weight', 'budget', 'height'],
      dtype='object')

Podemos ver que hay mucha informacion sobre los usuarios, te invito a tomar dos variables y explorar como estas se relacionan con la puntuacion promedio de cada persona, dejo a tu elección si quieres hacer un `set_index`, pero tienes que hacer al menos un `merge`, o dos si te sientes aventurero. Sean creativos! jueguen con los datos y vean a que pueden llegar. 

In [41]:
profile = profile.merge(park
                      , left_index = True #fill
                      , right_index = True #fill
                      , how = 'inner' #fill
                     )
profile.head()

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,...,dress_code,accessibility,price,Rambience,franchise,area,other_services,rating,food_rating,service_rating
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,...,informal,no_accessibility,medium,quiet,f,closed,Internet,1.000000,1.250000,0.750000
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,...,informal,completely,low,familiar,f,closed,none,1.181818,1.181818,1.000000
5,U1006,22.150000,-100.983000,true,social drinker,no preference,friends,car owner,single,independent,...,informal,partially,medium,familiar,f,open,none,1.200000,1.200000,1.200000
7,U1008,22.122989,-100.923811,false,social drinker,formal,solitary,public,single,independent,...,informal,completely,medium,familiar,t,closed,variety,0.857143,1.428571,0.857143
21,U1022,22.146708,-100.964355,false,casual drinker,formal,family,car owner,single,independent,...,informal,no_accessibility,low,quiet,f,closed,none,1.000000,1.166667,1.000000


**Conclusion**

Sé que lo que hicimos fue superficial, y que los datos dan para mucho más, así que en la proxima ayudantía, ya armados con más herramientas, espero que podamos volver a estos datos y hacer cositas más interesantes!